## Cleaning the data scraped from indeed.com

#### Steps taken:
1. add company-wide columns that pick bar pct for:
       Quick-paced
       Slow-paced
       Stressful
       Balanced
       Competitive
       Relaxed
       Cut-throat
2. add company-wide columns for ratings:
    Compensation/Benefits
    Culture
    Job Security
    Work/Life Balance
    Management

3. parse "width: 80%" to 1 - 5 and NaN for 0%
    b. add names to identify review-specific ratings

#### TO DO:

Load file

5. parse currently or not currently employed

6. analyze and remove duplicates 

4. parse company-wide attributes
    company_count_photos	
    company_count_reviews	
    company_count_salaries	
    company_empl	
    company_facebook	
    company_hq	
    company_indeed_url
   

In [1]:
import numpy as np
import pandas as pd
import re

In [10]:
file = "indeed_scrapy/scrape_data/company_reviews0to100fort20171020.csv"
df = pd.read_csv(file)


C:\Users\dsp21\Anaconda2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agg_rating,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,...,rating_scr3,rating_scr4,rating_scr5,review_date,review_title,reviewer_company_empl_status,reviewer_job_location,reviewer_job_title,top_list,work_life_rating
0,width: 40.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,3.4,3.2,3.1,"June 6, 2012",Good benefits.,(Former Employee) –,"Frisco, TX",HR Assistant & Account Analyst,Fortune 500 2017-03-31,width: 40.0%;
1,width: 80.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,3.4,3.2,3.1,"June 6, 2012",A day at AmerisourceBergen Bergen,(Current Employee) –,"Rockville, md",Program Rep I,Fortune 500 2017-03-31,width: 100.0%;
2,width: 100.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,3.4,3.2,3.1,"June 4, 2012",Clear objectives and diversity,(Current Employee) –,"Orange, California",Program Support Specialist,Fortune 500 2017-03-31,width: 60.0%;
3,width: 40.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,3.4,3.2,3.1,"May 30, 2012",it was a job and i would never work there again,(Former Employee) –,"Frisco, TX",NaN,Fortune 500 2017-03-31,width: 20.0%;
4,width: 60.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,3.4,3.2,3.1,"May 29, 2012",Productive and great benefits,(Current Employee) –,"Sugarland, TX",Associate 1,Fortune 500 2017-03-31,width: 40.0%;


In [104]:
print(df.shape)
df.head()


(126899, 67)


,agg_rating,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,...,Stressful,Balanced,Competitive,Relaxed,Cut-throat,Compensation/Benefits,Culture,Work/Life Balance,Management,Job Security/Advancement
0,width: 40.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
1,width: 80.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
2,width: 100.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
3,width: 40.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
4,width: 60.0%;,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1


1. creating columns for uncertain company ratings and bars

In [72]:
company_bar_score_names = ["Quick-paced", 'Slow-paced', 'Stressful', 'Balanced', 'Competitive', 'Relaxed', 'Cut-throat']
bar_dict = {'bar_name1' : 'bar_pct1', 
            'bar_name2' : 'bar_pct2', 
            'bar_name3' : 'bar_pct3', 
            'bar_name4' : 'bar_pct4', 
            'bar_name5' : 'bar_pct5'
}

company_rate_names = ["Compensation/Benefits", "Culture", "Job Security/Advancement", "Work/Life Balance", "Management"]
rate_dict = {'rating_name1' : 'rating_scr1', 
            'rating_name2' : 'rating_scr2', 
            'rating_name3' : 'rating_scr3', 
            'rating_name4' : 'rating_scr4', 
            'rating_name5' : 'rating_scr5'
}  

In [13]:
df['review_id'] = df.index
df['review_id'].head()

0    0
1    1
2    2
3    3
4    4
Name: review_id, dtype: int64

Need to assign the appropriate rating columns and then match them to the new columns to be created in df

In [32]:
test = bar_dict.keys()
type(bar_dict)
bar_dict[1]

KeyError: 1

In [36]:
#testing for loop
test = df #need to add .copy() to test without changing df
i = 2
mask = test['bar_name2'] == company_bar_score_names[i]

test.loc[mask,company_bar_score_names[i]] = test.loc[mask,'bar_pct2']
print(company_bar_score_names[i])
test[company_bar_score_names[i]].head(100)

Stressful


0     41%
1     41%
2     41%
3     41%
4     41%
5     41%
6     41%
7     41%
8     41%
9     41%
10    41%
11    41%
12    41%
13    41%
14    41%
15    41%
16    41%
17    41%
18    41%
19    41%
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
     ... 
70    NaN
71    NaN
72    NaN
73    NaN
74    NaN
75    NaN
76    NaN
77    NaN
78    NaN
79    NaN
80    NaN
81    NaN
82    NaN
83    NaN
84    NaN
85    NaN
86    NaN
87    NaN
88    NaN
89    NaN
90    NaN
91    NaN
92    NaN
93    NaN
94    NaN
95    NaN
96    NaN
97    NaN
98    NaN
99    NaN
Name: Stressful, Length: 100, dtype: object

In [85]:
df.filter(regex = '^b')

,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,bar_pct5
0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
1,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
2,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
3,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
4,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
5,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
6,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
7,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
8,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%
9,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,17%


In [93]:
#bar names
for name in company_bar_score_names:
    for bar, bar_pct in bar_dict.items():
        mask = df[bar].str.strip() == name
        df.loc[mask, name] = df.loc[mask, bar_pct]
        
    

In [97]:
#rating names
for name in company_rate_names:
    for rat, rat_scr in rate_dict.items():
        mask = df[rat].str.strip() == name
        df.loc[mask, name] = df.loc[mask, rat_scr]



In [102]:
df.iloc[:, -13:].head()

,review_id,Quick-paced,Slow-paced,Stressful,Balanced,Competitive,Relaxed,Cut-throat,Compensation/Benefits,Culture,Work/Life Balance,Management,Job Security/Advancement
0,0,47%,NaN,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
1,1,47%,NaN,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
2,2,47%,NaN,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
3,3,47%,NaN,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
4,4,47%,NaN,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1


In [109]:
df.filter(regex="^rat").isnull().apply(pd.value_counts)

,rating_name1,rating_name2,rating_name3,rating_name4,rating_name5,rating_scr1,rating_scr2,rating_scr3,rating_scr4,rating_scr5
False,126899,126899,126899,126899,126899,126899,126899,126899,126899,126899


Create functions for converting width to percent and string percent to float

In [113]:
def string_pct_to_float(strpct):
    return float(strpct.strip('%'))/100

def parse_widths_to_rating(string):
    
    try:
        result = string_pct_to_float(string.replace("^width: ","").replace(";","").strip())
        if result == 0:
            return np.nan
        else:
            return result * 5
        
        
print(string_pct_to_float('80%'))
print(parse_widths_to_rating('width: 100.0%'))

0.8
5.0


In [134]:
#replace widths with 
widths_to_replace_mask = df.iloc[0,:].str.contains("^width:") 

print(type(df.columns))
cols = df.columns[widths_to_replace_mask.fillna(False)]
df[cols] = df[cols].applymap(parse_widths_to_rating)




<class 'pandas.core.indexes.base.Index'>


,agg_rating,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,bar_pct1,bar_pct2,bar_pct3,bar_pct4,...,Stressful,Balanced,Competitive,Relaxed,Cut-throat,Compensation/Benefits,Culture,Work/Life Balance,Management,Job Security/Advancement
0,2.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
1,4.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
2,5.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
3,2.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
4,3.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
5,5.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
6,5.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
7,3.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
8,5.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1
9,4.0,Quick-paced,Stressful,Balanced,Competitive,Cut-throat,47%,41%,29%,25%,...,41%,29%,25%,NaN,17%,3.7,3.4,3.5,3.2,3.1


In [160]:
bar_names_count = df[list(bar_dict.keys())].apply(pd.Series.value_counts)
bar_names_count['totals'] = bar_names_count.sum(axis = 1)
bar_names_count = bar_names_count.append(pd.Series(bar_names_count.sum(axis=0), name = 'subtotals'))
bar_names_count

,bar_name1,bar_name2,bar_name3,bar_name4,bar_name5,totals
Balanced,11020.0,21320.0,57239.0,30220.0,6160.0,125959.0
Competitive,6819.0,17640.0,45240.0,45900.0,10200.0,125799.0
Cut-throat,NaN,100.0,860.0,7060.0,46660.0,54680.0
Quick-paced,97620.0,10420.0,9480.0,8439.0,NaN,125959.0
Relaxed,NaN,4099.0,200.0,8820.0,58240.0,71359.0
Slow-paced,NaN,NaN,NaN,NaN,80.0,80.0
Stressful,10500.0,72380.0,12940.0,25520.0,4619.0,125959.0
subtotals,125959.0,125959.0,125959.0,125959.0,125959.0,629795.0


In [169]:
val_cols = bar_names_count.index[:7]
#from above: df.filter(regex="^rat").isnull().apply(pd.value_counts)
val_col_counts = df[val_cols].isnull().apply(pd.value_counts)
val_col_counts['totals'] = val_col_counts.sum(axis = 1)
val_col_counts = val_col_counts.append(pd.Series(val_col_counts.sum(axis=0), name = 'subtotals'))
val_col_counts

,Balanced,Competitive,Cut-throat,Quick-paced,Relaxed,Slow-paced,Stressful,totals
False,125959,125799,54680,125959,71359,80,125959,629795
True,940,1100,72219,940,55540,126819,940,258498
subtotals,126899,126899,126899,126899,126899,126899,126899,888293


The values columns bnl check out with the others. I'll trust that the same goes for rating... it's late on Saturday and I don't have much time Sunday to work.

In [138]:
df.to_csv("indeed_scrapy/scrape_data/company_reviews0to100fort20171020" + "num_n_cowide sorted" + ".csv")